Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
%matplotlib inline

In [2]:
def load_mnist():
    """
    Load the MNIST dataset. Reads the training and testing files and create matrices.
    :Expected return:
    train_data:the matrix with the training data
    test_data: the matrix with the data that will be used for testing
    y_train: the matrix consisting of one 
                        hot vectors on each row(ground truth for training)
    y_test: the matrix consisting of one
                        hot vectors on each row(ground truth for testing)
    """
    
    #load the train files
    df = None
    
    y_train = []

    for i in range( 10 ):
        tmp = pd.read_csv( 'data/mnist/train%d.txt' % i, header=None, sep=" " )
        #build labels - one hot vector
        hot_vector = [ 1 if j == i else 0 for j in range(0,10) ]
        
        for j in range( tmp.shape[0] ):
            y_train.append( hot_vector )
            
        #concatenate dataframes by rows    
        if i == 0:
            df = tmp
        else:
            df = pd.concat( [df, tmp] )

    train_data = df.as_matrix()
    y_train = np.array( y_train )
    
    #load test files
    df = None
    
    y_test = []

    for i in range( 10 ):
        tmp = pd.read_csv( 'data/mnist/test%d.txt' % i, header=None, sep=" " )
        #build labels - one hot vector
        
        hot_vector = [ 1 if j == i else 0 for j in range(0,10) ]
        
        for j in range( tmp.shape[0] ):
            y_test.append( hot_vector )
            
        #concatenate dataframes by rows    
        if i == 0:
            df = tmp
        else:
            df = pd.concat( [df, tmp] )

    test_data = df.as_matrix()
    y_test = np.array( y_test )
    
    return train_data, test_data, y_train, y_test

In [3]:
def load_cifar():
    """
    Load the CIFAR-10 dataset. Reads the training and testing files and create matrices.
    :Expected return:
    train_data:the matrix with the training data
    test_data: the matrix with the data that will be used for testing
    y_train: the matrix consisting of one 
                        hot vectors on each row(ground truth for training)
    y_test: the matrix consisting of one
                        hot vectors on each row(ground truth for testing)
    """
    
    #load the train files
    
    y_train = []

    for i in range( 1, 6 ):
        tmp = open('data/cifar/data_batch_%d' % i, 'rb')
        
        train_dict = pickle.load(tmp, encoding='bytes')
        
        if i == 1:
            train_data = train_dict[b'data']
        else:
            train_data = np.vstack((train_data, train_dict[b'data']))
        
        #build labels - one hot vector
        
        for j in range( len(train_dict[b'labels'])):
            y_train.append([ 1 if k == train_dict[b'labels'][j] else 0 for k in range(0,10) ])

    y_train = np.array( y_train )
     
    y_test = []

    tmp = open('data/cifar/test_batch', 'rb')
        
    test_dict = pickle.load(tmp, encoding='bytes')
 
    test_data = test_dict[b'data']

    #build labels - one hot vector
                
    for j in range( len(test_dict[b'labels']) ):
        y_test.append( [ 1 if k == test_dict[b'labels'][j] else 0 for k in range(0,10) ] )

    y_test = np.array( y_test )
    
    return train_data, test_data, y_train, y_test

In [4]:
def load_data(Set=0):
    if Set==0:
        X_train, X_test, y_train, y_test = load_mnist()
    else:
        X_train, X_test, y_train, y_test = load_cifar()
    
    X_train = X_train.astype(float)/255
    X_test = X_test.astype(float)/255
    
    return X_train, X_test, y_train, y_test

Model

In [5]:
#use by default ax=1, when the array is 2D
#use ax=0 when the array is 1D
def softmax( x, ax=1 ):
    m = np.max( x, axis=ax, keepdims=True )#max per row
    p = np.exp( x - m )
    return ( p / np.sum(p,axis=ax,keepdims=True) )

In [6]:
#H : input array
#activation : integer for activation function selection
#returns an array modified by the activation function
def activation_function(H, activation): 
    
    if activation==1:
        z=np.log(1+np.exp(H))
        
    elif activation==2:
        z=(np.exp(H) - (np.exp(-H)))/(np.exp(H) + (np.exp(-H)))

    elif activation==3:
        z=np.cos(H)

    return z   

In [7]:
#H : input array
#activation : integer for activation function selection
#returns an array with the derivative of the activation function
def activation_derivative(H, activation):
    
    if activation==1:
        z=np.divide(np.exp(H),(1+np.exp(H)))
        
    elif activation==2:
        z=1-np.square((np.exp(H) - (np.exp(-H)))/(np.exp(H) + (np.exp(-H))))

    elif activation==3:
        z=-np.sin(H)

    return z

In [8]:
#H : input array
#returns an array where there is added a column to position 0 with all ones
def add_bias(Input):
    z = np.hstack((np.ones((Input.shape[0],1) ), Input))
    return z

In [9]:
#Input : input array
#W : weight array
#returns an array where each input value is multiplied with the corresponding weight
def feed_forward(Input,W):
    Input=add_bias(Input)
    Z=Input.dot(W.T)
    return Z

In [10]:
#Output : the output of a layer
#Input : input from previous layer
#W : weight array of a layer
#returns the gradient of W
def back_propagation(Output,Input,lamda,W):
    Input=add_bias(Input)
    grad=Output.T.dot(Input) - lamda*W
    return grad

In [11]:
#t : truth values
#y : imput multiplied by weights of the last layer
#lamda : regularization parameter
#W1,W2 : weights of first and second layer
#returns the cost for the values of W1 and W2
def cost_function(t,y,lamda,W1,W2):
    
    max_error = np.max(y, axis=1)
    Ew = np.sum(t * y) - np.sum(max_error) - \
    np.sum(np.log(np.sum(np.exp(y - np.array([max_error, ] * y.shape[1]).T), 1))) - \
    (0.5 * lamda) * (np.sum(np.square(W1))+np.sum(np.square(W2)))
    
    return Ew

In [12]:
#t : truth values
#X : imput values
#lamda : regularization parameter
#W1,W2 : weights of first and second layer
#activation : integer for activation function selection
#returns the cost for the values of W1 and W2
def cost_grads(t, X, lamda, W1, W2, activation):
    
    h = feed_forward(X,W1)
    z = activation_function(h,activation)
    y = feed_forward(z,W2)
    s = softmax(y)
    
    Ew = cost_function(t,y,lamda,W1,W2)

    pred=t-s
    # calculate gradients
    f=(pred.dot(W2[:,1:]))*activation_derivative(h, activation)    
    gradEw1=back_propagation(f,X,lamda,W1)
    gradEw2=back_propagation(pred,z,lamda,W2)

    return Ew,gradEw1,gradEw2

In [13]:
def train(t, X, W1init,W2init, options):
    """inputs :
      t: N x K one hot vector indicating the output classes
      X: N x D input data vector 
      W1init: M x D+1 dimensional vector of the initial values of the parameters
      W2init: K x M+1 dimensional vector of the initial values of the parameters
      options: options(1) is the maximum number of iterations
               options(2) is the tolerance
               options(3) is the learning rate eta
               options(4) is the lamda
               options(5) is the activation function to use
               options(6) is the batch size
               options(7) are the checkpoints at which should report error
                #iterations, tolerance, learning rate, lamda, activation, batch size, checkpoints

    outputs :
      W1,W2: the trained vectors of the parameters
      costs: the costs during training
      """
    
    W1 = W1init
    W2 =W2init

    # Maximum number of iteration of gradient ascend
    _iter = options[0]

    # Tolerance
    tol = options[1]

    # Learning rate
    eta = options[2]
    
    # Lamda
    lamda = options[3]
    
    # Activation to use
    activation = options[4]
    
    # Batch size
    batch_size = options[5]
    
    # Checkpoints
    checkpoints = options[6]

    Ewold = -np.inf
    costs = []
    for i in range( 1, _iter+1 ):
        batch_cost=[]
        for j in range(0,X.shape[0],batch_size):
            
            Ew,gradEw1,gradEw2 = cost_grads(t[j:j+batch_size,:], X[j:j+batch_size,:], lamda, W1, W2, activation)
            # save cost
            batch_cost.append(Ew)
            costs.append(Ew)
            # Break if you achieve the desired accuracy in the cost function
            if np.abs(Ew - Ewold) < tol:
                break
                # Update parameters based on gradient ascend
            W1 = W1 + eta * gradEw1
            W2 = W2 + eta * gradEw2
            
            Ewold = Ew
        if(i in checkpoints):
            report(W1,W2,i,options)
    return W1,W2, costs

In [14]:
#returns the prediction of the class based on weights, input and activation function
def predict(W1,W2, X_test,activation):
    
    h = feed_forward(X_test,W1)
    z = activation_function(h,activation)
    y = feed_forward(z,W2)
    s = softmax(y)

    prediction = np.argmax(s, 1)
    return prediction

In [15]:
#prints a report with the neural netowrk parameters and accuracy
def report(W1,W2,iteration,options):
    print("------------------------------------------------")
    print("Hidden Layers : " + str(W1.shape[0]))
    print("Activation Function Hidden Layer : " + str(options[4]))
    pred = predict(W1,W2, X_test,options[4])
    print("Accuracy on test dataset : "+str(np.mean( pred == np.argmax(y_test,1)) ))
    print("Iterations : "+ str(iteration))
    print("Learning rate : " + str(options[2]))
    print("Lamda : "+str(options[3]))
    print("Batch Size : "+str(options[5]))
    print("------------------------------------------------")

In [16]:
def gradcheck(W1init,W2init, X, t, lamda,activation):
    
    W1 = np.random.rand(*W1init.shape)
    W2 = np.random.rand(*W2init.shape)
    epsilon = 1e-6
    
    _list = np.random.randint(X.shape[0], size=5)
    x_sample = np.array(X[_list, :])
    t_sample = np.array(t[_list, :])
    
    Ew, gradEw1,gradEw2 = cost_grads(t_sample,x_sample, lamda,W1,W2,activation)

    numericalGrad1 = np.zeros(gradEw1.shape)
    numericalGrad2 = np.zeros(gradEw2.shape)
    
    for k in range(numericalGrad1.shape[0]):
        for d in range(numericalGrad1.shape[1]):
            
            #add epsilon to the w[k,d]
            w_tmp = np.copy(W1)
            w_tmp[k, d] += epsilon
            e_plus,_,_ = cost_grads(t_sample,x_sample, lamda,w_tmp,W2,activation)

            #subtract epsilon to the w[k,d]
            w_tmp = np.copy(W1)
            w_tmp[k, d] -= epsilon
            e_minus,_,_  = cost_grads(t_sample,x_sample, lamda,w_tmp,W2,activation)
            
            #approximate gradient ( E[ w[k,d] + theta ] - E[ w[k,d] - theta ] ) / 2*e
            numericalGrad1[k, d] = (e_plus - e_minus) / (2 * epsilon)
        
    
    for k in range(numericalGrad2.shape[0]):
        for d in range(numericalGrad2.shape[1]):
            
            #add epsilon to the w[k,d]
            w_tmp = np.copy(W2)
            w_tmp[k, d] += epsilon
            e_plus,_,_ = cost_grads(t_sample,x_sample, lamda,W1,w_tmp,activation)

            #subtract epsilon to the w[k,d]
            w_tmp = np.copy(W2)
            w_tmp[k, d] -= epsilon
            e_minus,_,_  = cost_grads(t_sample,x_sample, lamda,W1,w_tmp,activation)
            
            #approximate gradient ( E[ w[k,d] + theta ] - E[ w[k,d] - theta ] ) / 2*e
            numericalGrad2[k, d] = (e_plus - e_minus) / (2 * epsilon)
    

    return ( gradEw1,gradEw2, numericalGrad1,numericalGrad2 )

In [17]:
X_train, X_test, y_train, y_test = load_data()
# N of X
N, D = X_train.shape

K = 10

M = 100
np.random.seed(0)
# initialize w for the gradient ascent
W1init = np.random.randn(M, D+1)*0.01
W2init = np.random.randn(K, M+1)*0.01

# regularization parameter
lamda = 0.001

# options for gradient descent
options = [500, 1e-6, 10/N, 1]

C:\Users\Lefteris\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Lefteris\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [18]:
gradEw1,gradEw2, numericalGrad1,numericalGrad2 = gradcheck(W1init,W2init, X_train, y_train, lamda,options[3])

numerator1 = np.linalg.norm(gradEw1 - numericalGrad1)  
numerator2 = np.linalg.norm(gradEw2 - numericalGrad2)                                   

denominator1 = np.linalg.norm(gradEw1) + np.linalg.norm(numericalGrad1)  
denominator2 = np.linalg.norm(gradEw2) + np.linalg.norm(numericalGrad2)

difference = (numerator1 +numerator2)/ (denominator1  + denominator2)                                        # Step 3'

if difference > 1e-6:
    print("\033[93m" + "There is a mistake in the backward propagation! difference = " + str(difference) + "\033[0m")
else:
    print("\033[92m" + "Your backward propagation works perfectly fine! difference = " + str(difference) + "\033[0m")


Your backward propagation works perfectly fine! difference = 4.12289836112819e-08


In [19]:
X_train, X_test, y_train, y_test = load_data()
print("MNIST Dataset")
for i in range(100,400,100):
    print("***********************************************")
    for activation in range (1,4):
        N, D = X_train.shape
        K = 10

        M = i
        np.random.seed(0)
                        # initialize w for the gradient ascent
        W1init = np.random.randn(M, D+1)*0.01
        W2init = np.random.randn(K, M+1)*0.01

        iterations = 700 # number of epochs
        tolerance = 1e-6 # tolerance
        learning_rate = 10/N # learning rate
        lamda = 0.001 # regularization parameter    
        batch_size = 200  # batch size for gradient ascent
        checkpoints = [300,500,700] # epochs at which error will be reported
        
        # options for gradient descent
        #iterations, tolerance, learning rate, lamda, activation, batch size, checkpoints
        options = [iterations, tolerance, learning_rate, lamda, activation, batch_size, checkpoints]
        
        W1,W2, costs=train(y_train, X_train, W1init ,W2init, options)

C:\Users\Lefteris\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Lefteris\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


MNIST Dataset
***********************************************
------------------------------------------------
Hidden Layers : 100
Activation Function Hidden Layer : 1
Accuracy on test dataset : 0.9203
Iterations : 300
Learning rate : 0.00016666666666666666
Lamda : 0.001
Batch Size : 200
------------------------------------------------
------------------------------------------------
Hidden Layers : 100
Activation Function Hidden Layer : 1
Accuracy on test dataset : 0.9497
Iterations : 500
Learning rate : 0.00016666666666666666
Lamda : 0.001
Batch Size : 200
------------------------------------------------
------------------------------------------------
Hidden Layers : 100
Activation Function Hidden Layer : 1
Accuracy on test dataset : 0.961
Iterations : 700
Learning rate : 0.00016666666666666666
Lamda : 0.001
Batch Size : 200
------------------------------------------------
------------------------------------------------
Hidden Layers : 100
Activation Function Hidden Layer : 2
Accur

In [20]:
X_train, X_test, y_train, y_test = load_data(1)
print("CIFAR-10 Dataset")
for i in range(100,400,100):
    print("***********************************************")
    for activation in range (1,4):
        N, D = X_train.shape
        K = 10

        M = i
        np.random.seed(0)
                        # initialize w for the gradient ascent
        W1init = np.random.randn(M, D+1)*0.01
        W2init = np.random.randn(K, M+1)*0.01

        iterations = 250 # number of epochs
        tolerance = 1e-6 # tolerance
        learning_rate = 2/N # learning rate
        lamda = 0.01 # regularization parameter    
        batch_size = 100  # batch size for gradient ascent
        checkpoints = [150,200,250] # epochs at which error will be reported
        
        # options for gradient descent
        #iterations, tolerance, learning rate, lamda, activation, batch size, checkpoints
        options = [iterations, tolerance, learning_rate, lamda, activation, batch_size, checkpoints]
        
        W1,W2, costs=train(y_train, X_train, W1init ,W2init, options)

CIFAR-10 Dataset
***********************************************
------------------------------------------------
Hidden Layers : 100
Activation Function Hidden Layer : 1
Accuracy on test dataset : 0.4815
Iterations : 150
Learning rate : 4e-05
Lamda : 0.01
Batch Size : 100
------------------------------------------------
------------------------------------------------
Hidden Layers : 100
Activation Function Hidden Layer : 1
Accuracy on test dataset : 0.495
Iterations : 200
Learning rate : 4e-05
Lamda : 0.01
Batch Size : 100
------------------------------------------------
------------------------------------------------
Hidden Layers : 100
Activation Function Hidden Layer : 1
Accuracy on test dataset : 0.5051
Iterations : 250
Learning rate : 4e-05
Lamda : 0.01
Batch Size : 100
------------------------------------------------
------------------------------------------------
Hidden Layers : 100
Activation Function Hidden Layer : 2
Accuracy on test dataset : 0.5
Iterations : 150
Learning